## Setup

Mount google drive if on colab:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download modules:

In [3]:
!pip install langchain langchain_huggingface langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size

Imports:

In [4]:
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.schema.runnable import RunnableParallel

from langdetect import detect, DetectorFactory
import langdetect.lang_detect_exception


import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from concurrent.futures import ThreadPoolExecutor

We load the data below in the pipeline implementation.

Models:

We use the UBC Toucan model for translation:

To translate using Toucan models, we use the target language ISO-3 code as preix.

In [7]:
lang_names={
    "aar": "Afar",
    "ach": "Acholi",
    "afr": "Afrikaans",
    "aka": "Akan",
    "amh": "Amharic",
    "bam": "Bambara",
    "bas": "Basaa",
    "bem": "Bemba",
    "btg": "Bete Gagnoa",
    "eng": "English",
    "ewe": "Ewe",
    "fon": "Fon",
    "fra": "French",
    "hau": "Hausa",
    "ibo": "Igbo",
    "kbp": "Kabiye",
    "lgg": "Lugbara",
    "lug": "Luganda",
    "mlg": "Malagasy",
    "nyn": "Nyakore",
    "orm": "Oromo",
    "som": "Somali",
    "sot": "Sesotho",
    "swa": "Swahili",
    "tir": "Tigrinya",
    "yor": "Yoruba",
    "teo": "Ateso",
    "gez": "Geez",
    "wal": "Wolaytta",
    "fan": "Fang",
    "kau": "Kanuri",
    "kin": "Kinyawanda",
    "kon": "Kongo",
    "lin": "Lingala",
    "nya": "Chichewa",
    "pcm": "Nigerian Pidgin",
    "ssw": "Siswati",
    "tsn": "Setswana",
    "tso": "Tsonga",
    "twi": "Twi",
    "wol": "Wolof",
    "xho": "Xhosa",
    "zul": "Zulu",
    "nnb": "Nande",
    "swc": "Swahili Congo",
    "ara": "Arabic"
}


In [28]:
translation_model = HuggingFacePipeline.from_model_id(
    model_id="UBC-NLP/toucan-base",
    task="translation",
    pipeline_kwargs={
        "do_sample": True,
        "max_new_tokens": 300,
        "max_length": 512
    }
)

Device set to use cuda:0


For the rest of the pipeline, we use the IBM Granite model:

In [23]:
granite_model = HuggingFacePipeline.from_model_id(
    model_id="ibm-granite/granite-3.2-2b-instruct",
    task="text-generation",
    pipeline_kwargs={
        "return_full_text": False,
        "do_sample": False,
        "max_new_tokens": 300,
        "max_length": 512
    }
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 8.12 MiB is free. Process 287860 has 14.73 GiB memory in use. Of the allocated memory 14.60 GiB is allocated by PyTorch, and 8.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Language Detection & Translation

In [18]:
DetectorFactory.seed = 0

langdetect_to_toucan = {
    "en": "eng", "fr": "fra", "sw": "swa", "yo": "yor", "ha": "hau",
    "ig": "ibo", "ar": "ara", "xh": "xho", "zu": "zul", "am": "amh",
    "so": "som", "st": "sot", "rw": "kin", "lg": "lug", "ln": "lin",
    "sn": "tsn", "ss": "ssw", "ny": "nya", "mg": "mlg", "om": "orm",
    "ti": "tir", "nso": "tsn", "tn": "tsn", "ts": "tso", "tw": "twi",
    "wo": "wol", "kg": "kon", "ee": "ewe", "ff": "fan", "pcm": "pcm",
    "arq": "ara", "ffm": "fan", "kab": "kab", "bm": "bam", "nyn": "nyn"
}

def detect_language(text):
    try:
        detected_lang = detect(text)
        return langdetect_to_toucan.get(detected_lang, "unknown")
    except langdetect.lang_detect_exception.LangDetectException:
        return "unknown"

def detect_and_append_language(df, text_column="sentence"):
    df["detected_language"] = df[text_column].apply(detect_language)
    return df

Language detection chain:

In [19]:
def detect_language_chain(text):
    detected_lang = detect_language(text)
    return {"sentence": text, "detected_language": detected_lang}

Translation Framework:

In [21]:
translation_format_instructions = """
Return only a valid JSON object with the following keys:
{
  "translated_text": "The translated English text of the given input."
}
"""

translation_prompt = ChatPromptTemplate.from_template(
    """
    You are an expert in multilingual translation.
    Translate the following sentence into English using the given source language.

    Source Language: "{source_language}"
    Sentence: "{sentence}"

    {format_instructions}
    """
).partial(format_instructions=translation_format_instructions)

translation_chain = LLMChain(llm=translation_model, prompt=translation_prompt)

<ipython-input-21-b8a0201a24e9>:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  translation_chain = LLMChain(llm=translation_model, prompt=translation_prompt)


Translation if needed chain:

In [29]:
def process_language_translation(df, text_column="sentence"):
    df["detected_language"] = df[text_column].apply(detect_language)

    def translate_if_needed(row):
        if row["detected_language"] == "eng":
            return row[text_column]
        else:
            response = translation_chain.invoke(
                "source_language": row["detected_language"],
                "sentence": row[text_column]
            })
            return eval(response)["translated_text"]

    df["translated_text"] = df.apply(translate_if_needed, axis=1)
    return df

sample_df = m2_multilang_sentiment_df.sample(5, random_state=42)
processed_df = process_language_translation(sample_df)


Your input_length: 80 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


ValueError: The following `model_kwargs` are not used by the model: ['return_full_text'] (note: typos in the generate arguments will also show up in this list)

## Sentiment Analysis Setup

In [7]:
format_instructions = """
Return only a valid JSON object with the following keys:
{
  "sentiment": "positive" | "mixed" | "negative",
  "explanation": "A brief reason for this classification."
}
"""

sentiment_prompt = ChatPromptTemplate.from_template(
    """
    You are an expert in sentiment analysis.
    Analyze the following sentence and classify it as 'positive', 'negative', or 'mixed'.
    Then, explain your reasoning in 25 words or less.

    Sentence: "{sentence}"

    {format_instructions}
    """
).partial(format_instructions=format_instructions)

sentiment_chain = LLMChain(llm=granite_model, prompt=sentiment_prompt)

In [8]:
def analyze_sentiment(sentence):
    response = sentiment_chain.invoke({"sentence": sentence})
    parsed_response = json.loads(response["text"])
    return {"sentiment": parsed_response["sentiment"], "explanation": parsed_response["explanation"]}

## Toxicity Analysis Setup

In [6]:
toxic_format_instructions = """
Return a JSON object with the following keys:
- "toxicity": (toxic, non-toxic)
- "explanation": (a brief explanation for why the toxicity was classified this way)
Output only the JSON object. Do ot add extra characters or whitespace.
"""

toxicity_prompt = ChatPromptTemplate.from_template(
    """
    You are an expert in toxicity analysis.
    Analyze the following sentence and classify it as 'toxic'or 'non-toxic'.
    Then, explain your reasoning in 25 words or less.

    Sentence: "{sentence}"

    {toxic_format_instructions}
    """
).partial(toxic_format_instructions=toxic_format_instructions)

toxicity_chain = LLMChain(llm=granite_model, prompt=toxicity_prompt)

<ipython-input-6-83c1c0d8f246>:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  toxicity_chain = LLMChain(llm=granite_model, prompt=toxicity_prompt)


In [9]:
def analyze_toxicity(sentence):
    response = toxicity_chain.invoke({"sentence": sentence})

    try:
        parsed_response = json.loads(response["text"])  # Attempt to parse JSON
        return {
            "toxicity": parsed_response.get("toxicity", "unknown"),
            "explanation": parsed_response.get("explanation", "No explanation provided.")
        }
    except json.JSONDecodeError as e:
        print(f"JSON parsing failed for sentence: {sentence}\nError: {e}\nResponse: {response['text']}")
        return {"toxicity": "unknown", "explanation": "Malformed response."}  # Safe fallback

## Detoxification Setup

In [10]:
from langchain.prompts import ChatPromptTemplate
detox_format_instructions = """
Return only a valid JSON object with the following keys:
{
  "detoxified_text": "A rewritten version of the input sentence that removes toxicity while keeping the original intent."
}
"""

detox_prompt = ChatPromptTemplate.from_template(
    """
    You are an AI assistant that removes toxicity from text.
    Rewrite the following sentence to be non-toxic while preserving its original meaning.

    Toxic Sentence: "{sentence}"

    {format_instructions}
    """
).partial(format_instructions=detox_format_instructions)

detox_chain = LLMChain(llm=granite_model, prompt=detox_prompt)

In [18]:
import json
import re

def detoxify_text(row):
    if row["prediction"] == "toxic":
        response = detox_chain.invoke({"sentence": row["text"]})
        raw_text = response["text"].strip()

        if raw_text.startswith("```json") and raw_text.endswith("```"):
            raw_text = raw_text[7:-3].strip()

        raw_text = re.sub(r'[\x00-\x1F\x7F]', '', raw_text)

        try:
            parsed_response = json.loads(raw_text)
            return parsed_response.get("detoxified_text", row["text"])
        except json.JSONDecodeError as e:
            print(f"JSON parsing failed for detox: {row['text']}\nError: {e}\nResponse: {raw_text}")
            return row["text"]
    else:
        return row["text"]

## Running the data:

In [43]:
sentiment_test_path = "/content/drive/MyDrive/MDS/COLX_565/data/Milstone-2-multilingual-sentiment-test-solutions.csv"
m2_multilang_sentiment_df = pd.read_csv(sentiment_test_path)

sentiment_results = []
for sentence in m2_multilang_sentiment_df["sentence"]:
    try:
        response = analyze_sentiment(sentence)
        sentiment_results.append(response)
    except Exception as e:
        print(f"Error processing sentence: {sentence}\nError: {e}")
        sentiment_results.append({"sentiment": "unknown", "explanation": "Failed to process."})

m2_multilang_sentiment_df["prediction"] = [res["sentiment"] for res in sentiment_results]
m2_multilang_sentiment_df["explanation"] = [res["explanation"] for res in sentiment_results]

true_sentiment = m2_multilang_sentiment_df["class-label"]
pred_sentiment = m2_multilang_sentiment_df["prediction"]

precision, recall, f1, _ = precision_recall_fscore_support(true_sentiment, pred_sentiment, average="macro")
accuracy = accuracy_score(true_sentiment, pred_sentiment)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Error processing sentence: Ni kipaji gani! Hiki kipikiosafi kinafanya kazi poa sana, lakini inachukua muda mrefu kuwasha.
Error: Expecting value: line 2 column 1 (char 1)
Error processing sentence: Ninapenda ubora wa picha, ila simu hii inajifunga ghafla, sijui tatizo ni nini.
Error: Expecting value: line 2 column 1 (char 1)
Error processing sentence: Kweli, kozi hii ya mtandaoni inavutia, lakini ada zake ni kali mno, sijui kama inastahili.
Error: Expecting value: line 2 column 1 (char 1)
Error processing sentence: Nashukuru huduma ya haraka, lakini chakula kimefika kikiwa baridi, si tamu kamwe.
Error: Expecting value: line 2 column 1 (char 1)
Error processing sentence: Haya magari mapya ni mazuri, ila bei yake 'inaniwasha kichwa' vibaya sana.
Error: Expecting value: line 2 column 1 (char 1)
Error processing sentence: Nilifikiri itakuwa mbovu, kumbe app hii ya mazoezi inanifanya nihisi nipo gym halisi, nimeipenda!
Error: Expecting value: line 2 column 1 (char 1)
Error processing senten

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
toxicity_test_path = "/content/drive/MyDrive/MDS/COLX_565/data/Milestone-2-toxic-test-solutions.csv"
m2_toxicity_df = pd.read_csv(toxicity_test_path)


toxicity_results = m2_toxicity_df["text"].apply(analyze_toxicity)
m2_toxicity_df["prediction"] = toxicity_results.apply(lambda x: x["toxicity"])
m2_toxicity_df["explanation"] = toxicity_results.apply(lambda x: x["explanation"])

# compute evaluation metrics
true_toxicity = m2_toxicity_df["source_label"]
pred_toxicity = m2_toxicity_df["prediction"]

tox_precision, tox_recall, tox_f1, _ = precision_recall_fscore_support(true_toxicity, pred_toxicity, average="macro")
tox_accuracy = accuracy_score(true_toxicity, pred_toxicity)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


NameError: name 'true_sentiment' is not defined

In [13]:
tox_precision, tox_recall, tox_f1, _ = precision_recall_fscore_support(true_toxicity, pred_toxicity, average="macro")
tox_accuracy = accuracy_score(true_toxicity, pred_toxicity)

In [49]:
m2_multilang_sentiment_df.to_csv("sentiment_analysis_results.csv", index=False)
m2_toxicity_df.to_csv("toxicity_analysis_results.csv", index=False)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
print("Sentiment Analysis Evaluation:")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

Sentiment Analysis Evaluation:


NameError: name 'accuracy' is not defined

In [24]:
print("\nToxicity Detection Evaluation:")
print(f"Accuracy: {tox_accuracy:.3f} ")
print(f"Precision: {tox_precision:.3f}")
print(f"Recall: {tox_recall:.3f}")
print(f"F1 Score: {tox_f1:.3f}")


Toxicity Detection Evaluation:
Accuracy: 0.560 
Precision: 0.444
Recall: 0.479
F1 Score: 0.413


Detoxify:

In [20]:
m2_toxicity_df["detoxified_text"] = m2_toxicity_df.apply(detoxify_text, axis=1)

JSON parsing failed for detox: 2. 	> Lol, I guessed IPN (the Institue of National Remembrance) was going to be mentioned  From what I understood from their message on IPN's page, they actually went after that cunt that modified the trailer (and has two more cunty videos on his channel), not after the actual game itself.  The journalists however - especially the rightwing ones - seen IPN's message and went after everything. No surprise there, lol.  >  Germans are the last people who would make that mistake with the phrasing.  You would think so. Sadly, you would be wrong.  - http://wyborcza.pl/1,76842,6928930,_Polish_Camps__in_Polish_Court.html - https://fakty.interia.pl/news-byly-wiezien-auschwitz-skarzy-zdf-za-polskie-obozy,nId,999725
Error: Unterminated string starting at: line 1 column 23 (char 22)
Response: {  "detoxified_text": "2. > I anticipated IPN (the Institute of National Remembrance) would be referenced. Based on the information I gathered from IPN's page, it appears they t

In [21]:
# Display first few rows of the DataFrame
print(m2_toxicity_df[["text", "prediction", "detoxified_text"]].head())

                                                text prediction  \
0  I know he's an idiot I think he might of been ...      toxic   
1    ==my butthole==  i love buttholes. they tast...      toxic   
2  This is absolute bullshit! We fucking knew thi...      toxic   
3  Moochelle Nobama takes swipe at Trump it sez. ...      toxic   
4  2. \t> Lol, I guessed IPN (the Institue of Nat...      toxic   

                                     detoxified_text  
0  I suspect he may have faced some academic chal...  
1  I have a strong affinity for buttocks, finding...  
2  It's incredibly disappointing to see this situ...  
3  Mrs. Obama reportedly criticizes Trump, sugges...  
4  2. \t> Lol, I guessed IPN (the Institue of Nat...  
